In [1]:
import sys
import rospy
import moveit_commander


In [2]:
rospy.init_node('test_manipulation', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

group_name = robot.get_group_names()
group_name

['arm', 'gripper']

In [3]:
robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_footprint"
  name: 
    - joint1
    - joint2
    - joint3
    - joint4
    - grip_joint
    - grip_joint_sub
    - lb_base
    - lf_base
    - rb_base
    - rf_base
  position: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "base_footprint"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [24]:
move_group = moveit_commander.MoveGroupCommander(group_name[0])
move_group.set_named_target('home')
success, trajectory, _, _ = move_group.plan()

In [25]:
if success:
    move_group.execute(trajectory, wait=True)
else:
    print("Planning failed!")

In [32]:
def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_states = move_group.get_current_joint_values()
    print ("--> current joint state as follows (rad) :")
    print (joint_states)
    print ("--> current joint state as follows (degree) :")
    print ([joint*180./math.pi for joint in joint_states])

In [35]:
import math

get_joint_state(move_group)
degrees = [50., -0., 20., 50.]
goal = ([n*math.pi/180. for n in [50., -0., 40., 90.]])

move_joints(move_group, goal)
get_joint_state(move_group)

--> current joint state as follows (rad) :
[0.0, 0.0, 0.0, 0.0]
--> current joint state as follows (degree) :
[0.0, 0.0, 0.0, 0.0]
--> current joint state as follows (rad) :
[0.0, 0.0, 0.0, 0.0]
--> current joint state as follows (degree) :
[0.0, 0.0, 0.0, 0.0]


In [20]:
import geometry_msgs.msg #엔드이펙터 위치자세
pose_goal = geometry_msgs.msg.Pose()

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    return joint_state

pose_goal = get_goal_pose(move_group)
pose_goal

header: 
  seq: 0
  stamp: 
    secs: 3542
    nsecs: 739000000
  frame_id: "base_footprint"
pose: 
  position: 
    x: -0.045000000000000005
    y: 0.001
    z: 0.6779999999999999
  orientation: 
    x: 0.0
    y: 0.0
    z: 0.0
    w: 1.0

In [21]:
pose_goal.pose.orientation

x: 0.0
y: 0.0
z: 0.0
w: 1.0

In [22]:
import tf

quat_angle = tf.transformations.euler_from_quaternion([
    pose_goal.pose.orientation.x,
    pose_goal.pose.orientation.y,
    pose_goal.pose.orientation.z,
    pose_goal.pose.orientation.w
])

[each*180./math.pi for each in quat_angle]

[0.0, -0.0, 0.0]

In [23]:
[p, q, r] = [each*math.pi/180. for each in [0., 0., 30.]]

x, y, z, w = tf.transformations.quaternion_from_euler(p, q, r)

pose_goal.pose.orientation.x = x
pose_goal.pose.orientation.y = y
pose_goal.pose.orientation.z = z
pose_goal.pose.orientation.w = w


move_group.set_pose_target(pose_goal)
move_group.set_goal_orientation_tolerance(0.1)  # Adjust as needed
move_group.set_goal_position_tolerance(0.1)
move_group.set_planning_time(40.0) 
success = move_group.go(wait=True)
if success:
    rospy.loginfo("Move successful!")
    print("move suc")
else:
    rospy.logwarn("Move failed!")
    print("move failed")
move_group.stop()
move_group.clear_pose_targets()

move failed


[WARN] [1694411692.536465, 3582.943000]: Move failed!
